## Overview

#### This project is to analyse the house pricing in melbourne and related features

#### The objective of this notebook is to analyze the data of melbourne housing prices and its related features, and to be able to conclude the relationships in well performed model. To summarize, the goals of this notebook are to be concluded as two use cases:
>**_1. The model that explains the pricing by its related features_**

>**_2. The melbourne housing markets trend between 2016-2017._**

#### The goals are expected to meet common interests of certain stakeholders(Buyers&Sellers, Bank, Government, Agents and general public).
#### This notebook will perform data preprocession before stepping into the features analyzing and model building. 
#### With cleaned dataset, to achieve the goals, this notebook will examine features which obviously related to the housing prices. According to the domain knowledge, it will primarily focus on the the features that has obvious relathionshp with property price. Various visualization will be used for better presentation of data throughout the notebook during examining the relationship between these features and pricing. 
#### Features that will be explore are listed as follows:
>**_Price: The distribution of target feature_**

>**_Type: The house types included in Dataset & the impact of it on Price_**

>**_Location_**
>>**_Region: The regions' distribution and its impact on Price_**

>>**_Council: The co_**

>>**_Suburb_**

>>**_Distance: The distance of each case to city and the influence o_**

>**_Other features_**

>>**_Rooms: The room numbers and how the price spread with it_**

>>**_Bathrooms_**

>>**_Car_**

>>**_Landsize_**


#### This notebook is designed as sections of short and long answered questions, which helps to deduct the useful informations for the goals proposed above. 


In [ ]:
## Import library to support the analysis
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## _Section 1: Data preprocession_

#### Check the data(scan the head and tail)

In [ ]:
## Read in file
melb_house = pd.read_csv('file/melb_data.csv')
melb_house.head()

In [ ]:
melb_house.tail()

#### Check the datatype to understand the value in each column 

In [ ]:
## Check data info for each column
melb_house.info()

#### Do we have missing values in the data, if so, where are these missing values locating at?

In [ ]:
## Check missing values
melb_house.isnull().sum()

##### Based on our domain knowledge, the missing value in building area and built year requires further reference to be able to fill them, which is out of scope in the data available here. 

##### The council area can be filled with cross matching the full column 'Suburb'. This column can still be taken into consideration of pricing analysis if there is no more missing values after filled. 

#### Filling the missing values in CouncilArea

In [ ]:
pd.set_option('display.max_rows', 500)
melb_house.loc[melb_house['CouncilArea'].isna()][['Suburb','CouncilArea']]

In [ ]:
melb_house_ref = melb_house.groupby(['Suburb', 'CouncilArea'])['Address'].count().reset_index(name='count')    
## Check if there is 0 in the reference 
melb_house_ref['count'].unique()

In [ ]:
melb_house_ref.info()

In [ ]:
## transfer the dataframe to dictionary
dict_suburb = dict([(i, [x]) for i, x in zip(melb_house_ref['Suburb'], melb_house_ref['CouncilArea'])])

In [ ]:
## check the reference dictonary
print(dict_suburb)

In [ ]:
## check the column without string to fill later
def isNaN(string):
    return string != string

In [ ]:
## fill the empty cell with reference dictionary
for r, row in enumerate(melb_house['CouncilArea'].values):
    if isNaN(row) and melb_house['Suburb'][r] in dict_suburb:
        melb_house['CouncilArea'][r] = dict_suburb[melb_house['Suburb'][r]]

In [ ]:
### check the columns after filling
melb_house.loc[melb_house['CouncilArea'].isna()][['Suburb', 'CouncilArea']]

#### Is there still missing value in council area after filling?

In [ ]:
melb_house['CouncilArea'].isnull().sum()

#### What are the values in column car?

In [ ]:
## Check the values in Column Car
melb_house['Car'].unique()

In [ ]:
car = melb_house['Car'].reset_index()
car.groupby('Car').agg('count')

#### what the house type of the property that have missing car values?

In [ ]:
## prepared dataframe
car_na = melb_house[melb_house['Car'].isna()][['Type']].reset_index()

In [ ]:
car_na.groupby('Type').agg('count')

In [ ]:
## apply pie chart to visualize the house type without value in car column. 
car_grouped = car_na.groupby('Type').size()
car_grouped.plot(kind='pie', subplots = True, figsize=(8,8))
plt.title('Pie chart of the house type without value in column Car')
plt.ylabel('')
plt.show()

#### We can conclude that the missing values in car include property types of both unit and house. Based on the domain knowledge, the house has at least the street parking space, we can fill them as 1 car space. As for unit, no reference can be made without supporting data, we can use 0 to fill here as it is a very low portion in this data, also because the car space does not alter the pricing of unit much based on the domain knowledge. 

#### Filling the missing value in column 'Car'

In [ ]:
melb_house.loc[(melb_house['Type'] == 'h') & (melb_house['Car'].isna()), 'Car']=1
melb_house.loc[(melb_house['Type'] == 'u') & (melb_house['Car'].isna()), 'Car']=0

In [ ]:
## check the cells in 'car' after filling 
melb_house['Car'].isna().sum()

In [ ]:
melb_house.isnull().sum()

### Analysis

#### After basic information fill of the dataset. Now the column car is filled and only very small proportion of the council column is missing, which cannot be filled without further research on their councilArea. Fortunately, this dataset includes other features that is highly related to price based on domain knowledge, which can be analysed and involved in solving use cases. 

#### Next section will continue on exploring the available features and their relationship with price.

## _Section 2: Features analyzing_

### Price

##### How is price distributed?

In [ ]:
melb_house['Price'].describe()

In [ ]:
sns.set_theme(style="whitegrid")
sns.displot(data=melb_house, x="Price", kde=True)

##### It is right-skewed due to a few extremely high price properties.

In [ ]:
##subsetting the dataset to avoid the impact of outliers(extremely high price)
melb_before4m = melb_house[melb_house['Price']<=4000000].reset_index()
melb_after4m = melb_house[melb_house['Price']>4000000].reset_index()
melb_before3m = melb_house[melb_house['Price']<=3000000].reset_index()
melb_after3m = melb_house[melb_house['Price']>3000000].reset_index()

In [ ]:
## check the graph after 4m.
sns.displot(data=melb_after4m, x="Price", kde=True)

In [ ]:
## check the price distribution before 4m
sns.displot(data=melb_before4m, x="Price", kde=True)


##### The distribution of price still slightly right-skewed. Try 3m.

In [ ]:
## check the price distribution within 3m
sns.displot(data=melb_before3m, x="Price", kde=True)

In [ ]:
## check the price distribution after 3m
sns.displot(data=melb_after3m, x="Price", kde=True)
melb_after3m.info()

##### The distribution looks bell shape alike now. This notebook will continue discuss other features and their realtionship with price based on the subsets above.

### House Type

##### How many types there are and how are they distributed?

In [ ]:
melb_house.groupby(['Type'])['Type'].count().reset_index(name='count') 

In [ ]:
##visualize the distribution
sns.countplot(x=melb_house['Type'])

In [ ]:
melb_house.groupby('Type')['Price'].describe()

In [ ]:
sns.violinplot(data=melb_house, x="Type", y="Price",
               split=False, inner="quart", linewidth=1)

In [ ]:
melb_before3m.groupby('Type')['Price'].describe()

In [ ]:
sns.violinplot(data=melb_before3m, x="Type", y="Price",
               split=False, inner="quart", linewidth=1)

##### We can see from the graphs and tables above that there are obvious difference on price with different type of house. It is much more clear with the subset within 3m of the price. 

### Regionname

#### How does price distributed within different regions?

In [ ]:
melb_house['Regionname'].unique()

In [ ]:
### check the property sold based on regions
melb_before3m.groupby('Regionname')['Price'].describe()

In [ ]:
## visualize the price interval of different regions 
plt.figure(figsize = (10,10))
sns.catplot(data=melb_before3m, x="Price", y="Regionname", kind="box", palette="ch:.25")


##### As we can see from both graph and table, within different regions, price varies a lot as well as their mean, max and min values. Hence region has important impact on the price.

In [ ]:
## combine the type with region on price
sns.boxplot(x="Price", y="Regionname",
            hue="Type", palette=["m", "g", "r"],
            data=melb_before3m)

##### From the graph above, we can see the house type affect the price within different regions. Regions like northern victoria and western victoria only have type house here, and eastern victoria doesnot have townhouse. Their price and average of price vary significantly within same region or inter-regions. Hence region has huge impact on price.

### Council

##### How is Council Area distributed?

In [ ]:
## check the council area column
melb_before3m['CouncilArea'].describe()

In [ ]:
## check if there is missing value 
melb_before3m['CouncilArea'].isna().sum()

In [ ]:
## fill the missing value
melb_before3m['CouncilArea'].fillna('unknown', inplace=True)

In [ ]:
## check missing value after filling
melb_before3m['CouncilArea'].isna().sum()

##### How does council area spread on various regions?

In [ ]:
melb_council_before3m = melb_before3m.groupby(['CouncilArea', 'Regionname'])['Address'].count().reset_index(name='count')



In [ ]:
melb_before3m.groupby('CouncilArea').describe()

### Suburb

#### How is suburb distributed and how it impact on price?

In [ ]:
### prepre dataframe for visualisation
suburb_before3m= melb_before3m.groupby(['Suburb'])['Address'].count().reset_index(name='count')

suburb_before3m

##### since there are too many suburbs, we set threshold at 50 when visualizing.

In [ ]:
## visualize property sold based on different suburb
suburb_treemap = suburb_before3m[suburb_before3m['count']>50]
fig = px.treemap(suburb_treemap, path=['Suburb'], values='count') 
fig.show()

##### What are the top prices in different suburbs?

In [ ]:
df = melb_before3m.groupby('Suburb')['Price'].agg('max')
df

### Distance

##### How is distance distributed along price?


In [ ]:
melb_house['Distance'].describe()

In [ ]:
melb_before3m['Distance'].describe()

In [ ]:
sns.scatterplot(x="Distance", y="Price", hue='Regionname', data=melb_before3m)


In [ ]:
sns.lmplot(x="Distance", y="Price", data=melb_before3m)

##### The price is heavily concentrated on 0-20m in distance. It also varies a lot at the same distance, which is partialy contributed by different regions. Hence we conclude that distance alone does not make a useful predictor on price.

### Rooms

##### How is price spread along the room numbers?

In [ ]:
melb_before3m['Rooms'].unique()

In [ ]:
melb_before3m.groupby('Rooms')['Price'].describe()

##### Significantly small number of property after room number is greater than 6.

In [ ]:

sns.stripplot(y="Price", x="Rooms", hue = "Type", palette="deep", data=melb_before3m)


In [ ]:
sns.lmplot(x="Rooms", y="Price", hue = "Regionname", data=melb_before3m)

##### The patterns on both with type and regionname is generally fanning. Price is overall increasing along the room numbers. Moreover, each color shows different patterns. The analysis will carry on with subsets of different types of property.

In [ ]:
## subsetting dataset of prices within 3m
melb_house_3m = melb_before3m[melb_before3m['Type']=='h'].reset_index()
melb_unit_3m = melb_before3m[melb_before3m['Type']=='u'].reset_index()
melb_townhouse_3m = melb_before3m[melb_before3m['Type']=='t'].reset_index()

In [ ]:
## In type house set. 
sns.lmplot(x="Rooms", y="Price", data=melb_house_3m)


In [ ]:
sns.lmplot(x="Rooms", y="Price", data=melb_townhouse_3m)

### Bathrooms

### Car

In [ ]:
melb_house['Rooms'].unique()

In [ ]:
melb_house['Bedroom2'].unique()

In [ ]:
melb_house.groupby('Bedroom2')['Price'].describe()

In [ ]:
melb_house.groupby('Rooms')['Price'].describe()

### Landsize

In [ ]:
melb_house['Bathroom'].unique()

In [ ]:
melb_house.groupby('Bathroom')['Price'].describe()

### Method

In [ ]:
sns.set_theme(style="darkgrid")
sns.countplot(x=melb_house['Method'])

## _Section 3: Use Case 1- Features affecting Price_

## _Section 4: Use Case 2- Housing Market Trend(16-17)_